In [2]:
from rich import print
import pandas as pd
import numpy as np
import math
import ast
from dataclasses import dataclass
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
class Participant:
    def __init__(self, id, university, interests, preferred_role, friend_registration, preferred_team_size, availability,
                 programming_skills, interest_in_challenges, experience, languages_ordered, maturity, Tryhard, Rookie, Learner, Portfolio):
        self.id = id
        self.university = university
        self.interests = interests
        self.preferred_role = preferred_role
        self.friend_registration = friend_registration
        self.preferred_team_size = preferred_team_size
        self.availability = availability
        self.programming_skills = programming_skills
        self.interest_in_challenges = interest_in_challenges
        self.experience = experience
        self.languages_ordered = languages_ordered
        self.maturity = maturity
        self.Tryhard = Tryhard
        self.Rookie = Rookie
        self.Learner = Learner
        self.Portfolio = Portfolio

In [4]:
data_path = "data/datathon_participants.json"

In [5]:
def dist_university(university_1, university_2) :
    if university_1 == university_2:
        return 0 
    else:
        return 10

In [6]:
def dist_friends(friend_registration_1, friend_registration_2, id_1, id_2):
    if id_1 in friend_registration_2 and id_2 in friend_registration_1:
        return 0
    elif id_1 in friend_registration_2 or id_2 in friend_registration_1:
        return 1
    return 10

In [7]:
#No usar
def dist_age_year_of_study(age_1, age_2, year_1, year_2, age_weight, year_weigth) -> float:
    age_distance = abs(age_1 - age_2)
    year_distance = abs(year_1 - year_2)
    return (age_weight * age_distance + year_weigth * year_distance) / (age_distance + year_distance)


In [8]:
def dist_one_hot_encoding(x, y):
    return sum([1 if x_i != y_i else 0 for x_i, y_i in zip(x, y)])

In [9]:
def dist_programming_skills(skills_1, skills_2) -> float:
    avg_level_1 = sum(skills_1.values()) / len(skills_1) if skills_1 else 0
    
    avg_level_2 = sum(skills_2.values()) / len(skills_2) if skills_2 else 0
    
    distance_avg_levels = abs(avg_level_1 - avg_level_2)
    
    all_skills = set(skills_1.keys()).union(set(skills_2.keys()))
    
    distance_union = 1 / len(all_skills) if all_skills else 10 
    return distance_avg_levels + distance_union


In [10]:
#No usar
def dist_experience(exp_1, exp_2, hackathons_1, hackathons_2, experience_weights) -> float:
    exp_distance = abs(exp_1 - exp_2)
    hackathon_distance = abs(np.log1p(hackathons_1) - np.log1p(hackathons_2))
    total_distance = experience_weights[0] * exp_distance + experience_weights[1] * hackathon_distance
    return total_distance

In [11]:
def dist_preferred_role(role_1, role_2):
    if role_1 == role_2:
        return 1.0  

    if role_1 == "Don't know" or role_2 == "Don't know":
        return 1 / 0.2  

    if role_1 == "Don't care" or role_2 == "Don't care":
        return 0.5  

    return 0.0

In [12]:
#No usar
def euclidean_distance(vector_1, vector_2):
    squared_diff_sum = sum((x - y) ** 2 for x, y in zip(vector_1, vector_2))
    
    return math.sqrt(squared_diff_sum)

In [13]:
def dist_language(languages_1, languages_2):
    if not languages_1 or not languages_2:  
        return 0
    
    common_languages = set(languages_1).intersection(set(languages_2))
    
    if not common_languages:  
        return 100

    distance = 0
    for i, lang_1 in enumerate(languages_1):
        for j, lang_2 in enumerate(languages_2):
            if lang_1 == lang_2:
                d = abs(i - j)
                return d + math.exp(i+j) - 1

In [14]:
df = pd.read_csv('clean_data_actualitzat.csv')


In [15]:
participants_dict = {}

for _, row in df.iterrows():
    programming_skills = ast.literal_eval(row['programming_skills'])  
    interests = ast.literal_eval(row['interests']) 
    friend_registration = ast.literal_eval(row['friend_registration'])  
    availability = ast.literal_eval(row['availability'])  
    interest_in_challenges = ast.literal_eval(row['interest_in_challenges'])  
    languages_ordered = ast.literal_eval(row['languages_ordered'])  

    
    participant = Participant(
        id=row['id'],
        university=row['university'],
        interests=interests,  
        preferred_role=row['preferred_role'],
        friend_registration=friend_registration,  
        preferred_team_size=row['preferred_team_size'],
        availability=availability,  
        programming_skills=programming_skills,  
        interest_in_challenges=interest_in_challenges,  
        experience=row['experience'],
        languages_ordered=languages_ordered,  
        maturity=row['maturity'],
        Tryhard=row['Tryhard'],
        Rookie=row['Rookie'],
        Learner=row['Learner'],
        Portfolio=row['Portfolio']
    )
    
    participants_dict[row['id']] = participant

In [16]:
def get_min_max_values(participants_dict, weights):
    # Inicialitzar els valors mínims i màxims per cada atribut
    min_values = {
        'university': float('inf'),
        'interests': float('inf'),
        'preferred_role': float('inf'),
        'availability': float('inf'),
        'programming_skills': float('inf'),
        'interests_in_challenges': float('inf'),
        'languages': float('inf'),
        'experience': float('inf'),
        'maturity': float('inf'),
        'profile': float('inf'),
        'friends': float('inf')
    }

    max_values = {
        'university': float('-inf'),
        'interests': float('-inf'),
        'preferred_role': float('-inf'),
        'availability': float('-inf'),
        'programming_skills': float('-inf'),
        'interests_in_challenges': float('-inf'),
        'languages': float('-inf'),
        'experience': float('-inf'),
        'maturity': float('-inf'),
        'profile': float('-inf'),
        'friends': float('-inf')
    }

    # Iterar sobre tots els participants i calcular les distàncies
    participant_ids = list(participants_dict.keys())
    
    for i in range(len(participant_ids)):
        for j in range(i + 1, len(participant_ids)):  # Evitar calcular dues vegades la mateixa parella
            p1 = participants_dict[participant_ids[i]]
            p2 = participants_dict[participant_ids[j]]
            
            
            # Calcula les distàncies per atributs individuals
            d_university = dist_university(p1.university, p2.university) 
            d_interests = dist_one_hot_encoding(p1.interests, p2.interests) 
            d_preferred_role = dist_preferred_role(p1.preferred_role, p2.preferred_role) 
            d_availability = dist_one_hot_encoding(p1.availability, p2.availability) 
            d_programming_skills = dist_programming_skills(p1.programming_skills, p2.programming_skills) 
            d_interests_in_challenges = dist_one_hot_encoding(p1.interest_in_challenges, p2.interest_in_challenges) 
            d_languages = dist_language(p1.languages_ordered, p2.languages_ordered) 
            d_experience = abs(p1.experience - p2.experience) 
            d_maturity = abs(p1.maturity - p2.maturity) 
            d_profile = (abs(p1.Tryhard - p2.Tryhard) + abs(p1.Rookie - p2.Rookie) + abs(p1.Learner - p2.Learner) + abs(p1.Portfolio - p2.Portfolio)) 
            d_friend = dist_friends(p1.friend_registration, p2.friend_registration, p1.id, p2.id) 
            
            # Actualitzar min i max per a cada atribut
            min_values['university'] = min(min_values['university'], d_university)
            max_values['university'] = max(max_values['university'], d_university)

            min_values['interests'] = min(min_values['interests'], d_interests)
            max_values['interests'] = max(max_values['interests'], d_interests)

            min_values['preferred_role'] = min(min_values['preferred_role'], d_preferred_role)
            max_values['preferred_role'] = max(max_values['preferred_role'], d_preferred_role)

            min_values['availability'] = min(min_values['availability'], d_availability)
            max_values['availability'] = max(max_values['availability'], d_availability)

            min_values['programming_skills'] = min(min_values['programming_skills'], d_programming_skills)
            max_values['programming_skills'] = max(max_values['programming_skills'], d_programming_skills)

            min_values['interests_in_challenges'] = min(min_values['interests_in_challenges'], d_interests_in_challenges)
            max_values['interests_in_challenges'] = max(max_values['interests_in_challenges'], d_interests_in_challenges)

            min_values['languages'] = min(min_values['languages'], d_languages)
            max_values['languages'] = max(max_values['languages'], d_languages)

            min_values['experience'] = min(min_values['experience'], d_experience)
            max_values['experience'] = max(max_values['experience'], d_experience)

            min_values['maturity'] = min(min_values['maturity'], d_maturity)
            max_values['maturity'] = max(max_values['maturity'], d_maturity)

            min_values['profile'] = min(min_values['profile'], d_profile)
            max_values['profile'] = max(max_values['profile'], d_profile)

            min_values['friends'] = min(min_values['friends'], d_friend)
            max_values['friends'] = max(max_values['friends'], d_friend)

    return min_values, max_values


In [23]:
weights = {
    'university': 0.25,
    'interests': 0.25,
    'preferred_role': 0.5,
    'availability': 0.25,
    'programming_skills': 1,
    'interests_in_challenges': 1,
    'languages': 30,
    'experience': 1.5,
    'maturity': 0.5,
    'profile': 1,
    'friends' : 4

}


In [18]:
def combined_distance(participant1: Participant, participant2: Participant, weights: dict[str, float], min_values: dict, max_values: dict):
    # Calcular les distàncies per atribut
    d_university = dist_university(participant1.university, participant2.university)
    d_interests = dist_one_hot_encoding(participant1.interests, participant2.interests)
    d_preferred_role = dist_preferred_role(participant1.preferred_role, participant2.preferred_role)
    d_availability = dist_one_hot_encoding(participant1.availability, participant2.availability)
    d_programming_skills = dist_programming_skills(participant1.programming_skills, participant2.programming_skills)
    d_interests_in_challenges = dist_one_hot_encoding(participant1.interest_in_challenges, participant2.interest_in_challenges)
    d_languages = dist_language(participant1.languages_ordered, participant2.languages_ordered)
    d_experience = abs(participant1.experience - participant2.experience)
    d_maturity = abs(participant1.maturity - participant2.maturity)
    d_profile = (abs(participant1.Tryhard - participant2.Tryhard) + abs(participant1.Rookie - participant2.Rookie) + abs(participant1.Learner - participant2.Learner) + abs(participant1.Portfolio - participant2.Portfolio))
    d_friend = dist_friends(participant1.friend_registration, participant2.friend_registration, participant1.id, participant2.id)

    # Normalitzar les distàncies utilitzant els mínims i màxims
    d_university_normalized = (d_university - min_values['university']) / (max_values['university'] - min_values['university']) if max_values['university'] > min_values['university'] else 0
    d_interests_normalized = (d_interests - min_values['interests']) / (max_values['interests'] - min_values['interests']) if max_values['interests'] > min_values['interests'] else 0
    d_preferred_role_normalized = (d_preferred_role - min_values['preferred_role']) / (max_values['preferred_role'] - min_values['preferred_role']) if max_values['preferred_role'] > min_values['preferred_role'] else 0
    d_availability_normalized = (d_availability - min_values['availability']) / (max_values['availability'] - min_values['availability']) if max_values['availability'] > min_values['availability'] else 0
    d_programming_skills_normalized = (d_programming_skills - min_values['programming_skills']) / (max_values['programming_skills'] - min_values['programming_skills']) if max_values['programming_skills'] > min_values['programming_skills'] else 0
    d_interests_in_challenges_normalized = (d_interests_in_challenges - min_values['interests_in_challenges']) / (max_values['interests_in_challenges'] - min_values['interests_in_challenges']) if max_values['interests_in_challenges'] > min_values['interests_in_challenges'] else 0
    d_languages_normalized = (d_languages - min_values['languages']) / (max_values['languages'] - min_values['languages']) if max_values['languages'] > min_values['languages'] else 0
    d_experience_normalized = (d_experience - min_values['experience']) / (max_values['experience'] - min_values['experience']) if max_values['experience'] > min_values['experience'] else 0
    d_maturity_normalized = (d_maturity - min_values['maturity']) / (max_values['maturity'] - min_values['maturity']) if max_values['maturity'] > min_values['maturity'] else 0
    d_profile_normalized = (d_profile - min_values['profile']) / (max_values['profile'] - min_values['profile']) if max_values['profile'] > min_values['profile'] else 0
    d_friend_normalized = (d_friend - min_values['friends']) / (max_values['friends'] - min_values['friends']) if max_values['friends'] > min_values['friends'] else 0

    # Multiplicar les distàncies normalitzades pels pesos corresponents
    d_university_weighted = d_university_normalized * weights['university']
    d_interests_weighted = d_interests_normalized * weights['interests']
    d_preferred_role_weighted = d_preferred_role_normalized * weights['preferred_role']
    d_availability_weighted = d_availability_normalized * weights['availability']
    d_programming_skills_weighted = d_programming_skills_normalized * weights['programming_skills']
    d_interests_in_challenges_weighted = d_interests_in_challenges_normalized * weights['interests_in_challenges']
    d_languages_weighted = d_languages_normalized * weights['languages']
    d_experience_weighted = d_experience_normalized * weights['experience']
    d_maturity_weighted = d_maturity_normalized * weights['maturity']
    d_profile_weighted = d_profile_normalized * weights['profile']
    d_friend_weighted = d_friend_normalized * weights['friends']

    # Sumar les distàncies ponderades
    total_distance = (
        d_university_weighted +
        d_interests_weighted +
        d_preferred_role_weighted +
        d_availability_weighted +
        d_programming_skills_weighted +
        d_interests_in_challenges_weighted +
        d_languages_weighted +
        d_experience_weighted +
        d_maturity_weighted +
        d_profile_weighted +
        d_friend_weighted
    )
    return total_distance


In [19]:
def calculate_distance_matrix(participants_dict, weights, min_values, max_values):
    # Llista de tots els participants
    participant_ids = list(participants_dict.keys())
    num_participants = len(participant_ids)

    # Inicialitzar la matriu de distàncies amb zeros
    distance_matrix = np.zeros((num_participants, num_participants))

    # Recorre cada parell de participants
    for i in range(num_participants):
        for j in range(i + 1, num_participants):  # Comença a j=i+1 per evitar duplicats i càlculs innecessaris
            participant1 = participants_dict[participant_ids[i]]
            participant2 = participants_dict[participant_ids[j]]

            # Calcula la distància normalitzada
            distance = combined_distance(participant1, participant2, weights, min_values, max_values)
            
            # Omple la matriu de distàncies (simètrica)
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance

    return distance_matrix, participant_ids

In [24]:
min_values, max_values = get_min_max_values(participants_dict, weights)

matrix, ids = calculate_distance_matrix(participants_dict, weights, min_values, max_values)

# Imprimir la matriu de distàncies
for i, id1 in enumerate(ids):
    for j, id2 in enumerate(ids):
        print(f"Distància entre {id1} i {id2}: {matrix[i, j]:.2f}")


Distància entre 2ebad15c-c0ef-4c04-ba98-c5d98403a90c i 2ebad15c-c0ef-4c04-ba98-c5d98403a90c: 0.00

Distància entre 2ebad15c-c0ef-4c04-ba98-c5d98403a90c i a00428a2-14a2-4fb1-838a-22394778ec88: 6.75

Distància entre 2ebad15c-c0ef-4c04-ba98-c5d98403a90c i ab4b5790-0e6b-43db-bbaa-8670554f0cc8: 6.73

Distància entre 2ebad15c-c0ef-4c04-ba98-c5d98403a90c i cda2fafe-4b1e-4581-9fe0-1fb6d3e79aa3: 5.85

Distància entre 2ebad15c-c0ef-4c04-ba98-c5d98403a90c i 624f4677-d31a-4ecf-a7de-5df7392f9579: 6.43

...

Distància entre cda2fafe-4b1e-4581-9fe0-1fb6d3e79aa3 i 09d48185-f784-49bc-a8f8-406c1ca4db35: 7.63

Distància entre cda2fafe-4b1e-4581-9fe0-1fb6d3e79aa3 i d6b83b60-d69d-45b3-bcf2-fcadb1ed95c4: 6.00

Distància entre cda2fafe-4b1e-4581-9fe0-1fb6d3e79aa3 i 70876556-1071-42f9-9be4-a8e7af68e148: 6.34

Distància entre cda2fafe-4b1e-4581-9fe0-1fb6d3e79aa3 i e64f487a-7062-44bd-822a-4fcf8011b7e0: 6.51

KeyboardInterrupt: 

In [34]:
participant1 = participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c']
participant2 = participants_dict['52962082-92f1-42a2-91e0-6a2f1d320739']
#print(combined_distance(participant1, participant2, weights, min_values, max_values))
#print(dist_university(participant1.university, participant2.university))
#print(dist_one_hot_encoding(participant1.interests, participant2.interests))
#print(dist_preferred_role(participant1.preferred_role, participant2.preferred_role))
#print(dist_one_hot_encoding(participant1.availability, participant2.availability))
#print(dist_programming_skills(participant1.programming_skills, participant2.programming_skills))
#print(dist_one_hot_encoding(participant1.interest_in_challenges, participant2.interest_in_challenges))
#print(dist_language(participant1.languages_ordered, participant2.languages_ordered))
#print(abs(participant1.experience - participant2.experience))
print(abs(participant1.maturity - participant2.maturity))
participant1.maturity,participant2.maturity
#print((abs(participant1.Tryhard - participant2.Tryhard) + abs(participant1.Rookie - participant2.Rookie) + abs(participant1.Learner - participant2.Learner) + abs(participant1.Portfolio - participant2.Portfolio)))
#print(dist_friends(participant1.friend_registration, participant2.friend_registration, participant1.id, participant2.id))

nan

(4.6, nan)